In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.integrate import quad
from scipy.integrate import solve_ivp
from ipywidgets import interact


def hune_system(f_list, a, b, initial_values, n_grid_points):
    n_grid_points = int(n_grid_points)
    h = (b - a) / n_grid_points
    dim = len(f_list)
    w = np.zeros((n_grid_points + 1, dim))
    t_values = np.linspace(a, b, n_grid_points + 1)
    w[0, :] = initial_values

    for i in range(n_grid_points):
        ti = t_values[i]
        k1 = np.array([f(ti, *w[i, :]) for f in f_list])
        k2 = np.array([f(ti + h / 3, *(w[i, :] + (h / 3) * k1)) for f in f_list])
        k3 = np.array([f(ti + (2 / 3) * h, *(w[i, :] + (2 / 3) * h * k2)) for f in f_list])

        w[i + 1, :] = w[i, :] + (h / 4) * (k1 + 3 * k3)

    return t_values, w



def sloshing(t, y, epsilon, lam, Omega):
  y1, y2 = y
  dy1_dt = y2
  dy2_dt = - (1 + epsilon * lam * Omega**2 * np.cos(Omega * t)) * (y1 - (epsilon**2 / 6) * y1**3)

  return [dy1_dt, dy2_dt]

def sloshing_functions(epsilon, lam, omega):

  def y_1_prime(t, y1, y2):
    return y2

  def y_2_prime(t, y1, y2):
    return - (1 + epsilon * lam * omega**2 * np.cos(omega * t)) * (y1 - (epsilon**2 / 6) * y1**3)

  return [y_1_prime, y_2_prime]


t_span = (0.0, 6.0)
#Valores iniciales y_1(0) = 0.0 y y_2(0) = 0.1
y0 = [0.0, 0.1]
t_eval = np.linspace(t_span[0], t_span[1], 1000)

epsilon2, lam2, Omega2 = 0.5, 1, 2

sol2 = solve_ivp(sloshing, t_span, y0, method='RK45' ,args=(epsilon2, lam2, Omega2), dense_output=True)
y1_sol2 = sol2.sol(t_eval)[0]
y2_sol2 = sol2.sol(t_eval)[1]
sols = [y1_sol2, y2_sol2]


def plot_hune_solution(t_values, w):
    num_functions = w.shape[1]
    fig, axes = plt.subplots(num_functions, 1, figsize=(10, 6))

    if num_functions == 1:
        axes = [axes]

    for i, ax in enumerate(axes):
        cs = CubicSpline(t_values, w[:, i])
        t_fine = np.linspace(t_values[0], t_values[-1], 1000)
        w_fine = cs(t_fine)
        ax.plot(t_fine, w_fine, label=f'$y_{i+1}(t)$', color='orange')
        ax.plot(t_eval, sols[i], label=r'$\varepsilon=0.5,\ \lambda=1,\ \Omega=2$', color ='gray', linestyle = 'dashed')
        plt.subplots_adjust(hspace=0.4)  # Aumenta el espacio vertical
        ax.set_xlabel('Tiempo escalado ($\omega$t)')
        ax.set_ylabel(f'$y_{i+1}(t)$')
        ax.set_title(f'Solución para $y_{i+1}(t)$')
        ax.legend()
        ax.grid()

epsilon1, lam1, Omega1 = 0.1, 0.3, 1
t_span = (0.0, 6.0)
#Valores iniciales y_1(0) = 0.0 y y_2(0.1) = 0.1
y0 = [0.0, 0.1]


In [ ]:
def interactive_plot(tam_malla):
    tam_malla= int(tam_malla)
    t_values, w = hune_system(sloshing_functions(0.5, 1, 2), 0, 6, [0, 0.1], tam_malla)
    plot_hune_solution(t_values, w)

interact(interactive_plot, tam_malla=(1, 20, 1))

interactive(children=(IntSlider(value=10, description='tam_malla', max=20, min=1), Output()), _dom_classes=('w…

<function __main__.interactive_plot(tam_malla)>